# Mask Cover Percentage Score

In [1]:
import ee
ee.Initialize()

In [2]:
from geetools import ui, tools, cloud_mask, collection

In [3]:
from geebap import scores
from geebap.season import Season

In [4]:
p = ee.Geometry.Point([-71.1, -42.8])
aoi = p.buffer(8000).bounds()

In [5]:
col = collection.Landsat8SR()

In [6]:
colEE = col.collection.filterBounds(aoi).map(lambda img: img.addBands(col.ndvi(img)))

In [7]:
year = 2018

In [8]:
start = '11-15'
end = '02-15'

In [9]:
season = Season(start, end)

In [10]:
season_y = season.add_year(year)

In [11]:
ui.eprint(season_y)

'2017-11-15T00:00:00 to 2018-02-15T00:00:00'



In [12]:
colEE = colEE.filterDate(season_y.start(), season_y.end())

In [13]:
ui.eprint(colEE.size())

16



## Apply Cloud Mask

In [14]:
col.bitOptions()

{'pixel_qa': ['high_confidence_cirrus',
  'shadow',
  'high_confidence_cloud',
  'clear',
  'cloud',
  'water',
  'snow'],
 'radsat_qa': ['B1_saturated',
  'B2_saturated',
  'B3_saturated',
  'B4_saturated',
  'B5_saturated',
  'B6_saturated',
  'B7_saturated',
  'B9_saturated',
  'B10_saturated',
  'B11_saturated'],
 'sr_aerosol': ['water', 'climatology', 'low', 'medium', 'high']}

In [15]:
colEE = colEE.map(lambda img: col.applyMask(img, 'pixel_qa', ['cloud', 'shadow', 'snow']))

In [16]:
masked_i = ee.Image(colEE.first())

## Make pixels value zero in a random zone

In [17]:
p_zeros = ee.Geometry.Point([-71.11656188964845, -42.820320451492094]).buffer(1000)

In [18]:
col_zeros = colEE.map(lambda img: img.where(ee.Image.constant(1).clip(p_zeros), 0))

## Apply Score

In [19]:
score = scores.MaskPercentKernel()

In [20]:
col_score = score.map(col_zeros)

In [21]:
score_i = ee.Image(col_score.first())

## MAP

In [22]:
vis = col.visualization('NSR')

In [23]:
Map = ui.Map()

In [24]:
Map.show()

Map(basemap={'max_zoom': 19, 'attribution': 'Map data (c) <a href="https://openstreetmap.org">OpenStreetMap</a…

In [25]:
# Map.addImageCollection(colEE_score, vis, nametags=[score.name])

In [26]:
Map.addLayer(aoi, None, 'AOI')

In [27]:
Map.addLayer(masked_i, vis, 'Masked Image')

In [28]:
Map.centerObject(masked_i)

In [29]:
Map.addLayer(score_i, vis, 'Image with score without counting zeros')

## Take zeros in count

In [30]:
score_zeros = scores.MaskPercent(count_zeros=True)

In [31]:
col_score_zeros = score_zeros.map(col_zeros, col=col, geom=aoi)

In [32]:
score_zeros_i = ee.Image(col_score_zeros.first())

In [33]:
Map.addLayer(score_zeros_i, vis, 'Image with scores taking zeros in count')